<a href="https://colab.research.google.com/github/davidsjohnson/wise24_xai_ac/blob/setup/notebooks/tutorial_image_xai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment and run this cell to use Google Colab
# !git clone https://github.com/davidsjohnson/wise24_xai_ac.git
# fix xgboost incompatiblity issue
!pip uninstall -y -q scikit-learn
!pip install -q scikit-learn==1.5.2

In [2]:
import sys
import os
# sys.path.append(os.path.realpath('../'))  # uncomment this line if you are running this notebook locally
sys.path.append(os.path.realpath('wise24_xai_ac')) # uncomment this line if you are running this notebook on Google Colab

In [3]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from scipy.stats import randint, uniform

import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn

import seaborn as sns
import matplotlib.pyplot as plt

from skimage import io

import utils
import img_utils
import models
import evaluate

# Data Loading

In [15]:
colab = True # change to False if running locally

base_dir = Path('../data/') if not colab else Path('wise24_xai_ac/data/')
# Full data from training and evaluation
train_csv = base_dir / 'affectnet_aus/train_aus.csv'
val_csv = base_dir / 'affectnet_aus/val_aus.csv'

# load training and validation data as pandas dataframes
# action units extracted from the AffectNet dataset via OpenFace
df_train = pd.read_csv(train_csv)
df_val = pd.read_csv(val_csv)

# smaller dataset for explanations
xai_csv = base_dir / 'affectnet_aus/eval_aus.csv'
df_xai = pd.read_csv(xai_csv)
df_xai['image'] = df_xai['image'].str.replace('../data', str(base_dir))

# get only the columns storing action units from the dataframe
# there are also facial landmarks and other features in the dataset could be useful
# but we igore them for now and focus on action units
feature_names = [col for col in df_val.columns if col.startswith('AU')]
categorical_features = [feat for feat in feature_names if '_c' in feat]
categorical_idxs = [i for i, feat in enumerate(feature_names) if '_c' in feat]

# setup categorical features for XGBoost
for feat in categorical_features:
    df_train[feat] = df_train[feat].astype('category')
    df_val[feat] = df_val[feat].astype('category')

# get the class labels
class_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']  # same class labels as before

# Gets all images from folder used for XAI tasks
images = [io.imread(f) for f in df_xai.image]

# XAI for Tabular Data

In [5]:
from xgboost import XGBClassifier

In [16]:
X_train = df_train[feature_names]
y_train = df_train['class']
X_test = df_val[feature_names]
y_test = df_val['class']

print('Training data shape:', X_train.shape, y_train.shape)
print('Test data shape:', X_test.shape, y_test.shape)

Training data shape: (36685, 35) (36685,)
Test data shape: (3908, 35) (3908,)


In [ ]:
# # Train model from scratch
random_state = 10
clf = XGBClassifier(booster='gbtree', enable_categorical=True, max_depth=15, eta=0.1, reg_lambda=30, random_state=random_state)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)

In [ ]:
# get model predictions
y_test_preds = clf.predict(X_test)
y_test_true = y_test

# eval results
print(classification_report(y_test_true, y_test_preds, target_names=class_names))

In [10]:
cm_data = confusion_matrix(y_test_true, y_test_preds)
cm = pd.DataFrame(cm_data, columns=class_names, index=class_names)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

NameError: name 'y_test_true' is not defined

# XAI for CNNs

In [ ]:
# Setup Data from AffectNet Deep Learning Model
TRAIN_MEAN = [0.485, 0.456, 0.406]
TRAIN_STD = [0.229, 0.224, 0.225]

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=TRAIN_MEAN, std=TRAIN_STD),
    transforms.Resize((224, 224))
])

data_dir = Path('../data/affectnet/val_class')
dataset = datasets.ImageFolder(root=data_dir, transform=test_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
# download checkpoint
ckpt_link = 'https://uni-bielefeld.sciebo.de/s/0tAa2wPhGxSDjbM/download'
ckpt_path = utils.download_file(ckpt_link,
                                'affectnet.pth',
                                cache_dir='../data/affectnet/model',
                                extract=False,
                                force_download=False
                                )
ckpt_path

File already exists at: ../data/affectnet/model/affectnet.pth


PosixPath('../data/affectnet/model/affectnet.pth')

In [ ]:
model = models.ResNet18(n_classes=len(class_names), pretrained=True)
model.load_state_dict(torch.load(ckpt_path, map_location='cpu'))

/Users/djohnson/micromamba/envs/wise24_xai_ac/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


<All keys matched successfully>

In [ ]:
inverse_weights = torch.from_numpy(1.0/np.array([74874, 134415, 25459, 14090, 6378, 3803, 24882, 3750])).type(torch.float32)
loss = torch.nn.CrossEntropyLoss(weight=inverse_weights)
_, _, preds, probs = evaluate.evaluate_model(model, dataloader, loss, device='cpu')

Evaluation Loss: 1.2141, Evaluation Accuracy: 0.5875
